Code written in Oct 2021 by Yuhan Wang
only suitable for UFMs when TESes are in normal stage
instead of fitting to noise model, this takes median noise from 5Hz to 50Hz
different noise levels here are based on phase 2 noise target and noise model after considering johnson noise at 100mK

Adapted to an ipython notebook by Caleb Wheeler October 27, 2021

# Imports/Setup
Not all imports are required, some are here for convince.

In [ ]:
# selects a non-interactive plotting backend
import matplotlib
matplotlib.use('Agg')

# required imports and favorite packages.
import pysmurf.client
import argparse
import numpy as np
import os
import sys
import time
import glob

import numpy as np
from scipy.interpolate import interp1d
import argparse
import time
import csv
import scipy.signal as signal
import matplotlib.pyplot as plt

# Here we append this notebook's directory to the paths the python uses to look for imports.
# This is a temporary measure used as demonstration and testing tool.

In [ ]:
# Here we append this notebook's directory to the paths the python uses to look for imports.
# his is a temporary measure used as demonstration and testing tool.
basedir_this_notebook = os.path.basename(__file__)
sys.path.append(basedir_this_notebook)

# LoadS - Getting control of the SMuRF blade by slot number
This is a class that can load and operate on an abstract number of SMuRF controllers
obtained by slot number.

In [ ]:
from operators.controler import LoadS

# SingleBand - Set up a single band
There are a lot of ways to set up or configure a single band (bias line). This class stores
the fundamental methods for setting up a given band. Some methods in this class script these
fundamental methods. The methods at the end of this class are more fundamental while the methods
at the beginning are more complex scrips.

In [ ]:
from operators.smurf_band import SingleBand

# TimeStreamData
### A class for acquiring and viewing time stream data.
Each instance of this class can acquire time streams, however each instance is only
used to view a single time stream at a time. For viewing multiple time streams,
initiate multiple instances of this class.

In [ ]:
from operators.time_stream import TimeStreamData

# Grouped Bands
Classes that can be any set of bands on a given SMuRF slot.

`GroupedBiases`is for doing simple commands that are the same across all smurf bands.

`AutoTune`allows each band to be individually set, with the idea of maximizing tuning across
bands, with the least number of timestreams. `AutoTune.time_stream_data` is an instance of the
TimeStreamData class that is used automatically to acquire data.

In [ ]:
from operators.bias_group import GroupedBiases, AutoTune

# The Script Example
All the classes and import statements above only set the stage, this where the dance begins!
## Example Parameters
These are parameters chosen to emulate the functionality of ufm_optimize_quick_normal.py.
However, several parameters have been abstracted in the classes their methods.
See the doc-strings in the various classes for more information about use cases.

In [ ]:
# example parameters
band = 4
slot_num = 2
bias_group = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
stream_time = 20
nperseg = 2**12
verbose = True

## Load the SMuRF controller
The load_s class instance can support multiple controllers, accessible by slot number in the
dictionaries load_s.cfg_dict, load_s.S_dict, and load_s.log_dict.

Initialization is automatic, see the doc-strings in the 'LoadS' class for details.

In [ ]:
# load a single S, or SMuRF controller instance for a given slot number
load_s = LoadS(slot_nums=[slot_num], verbose=verbose)
cfg = load_s.cfg_dict[slot_num]
S = load_s.S_dict[slot_num]

## Lock and Configure on a Single Band
Setup for locking on a single `band`. When auto_startup is True, the method SingleBand.startup()
handles all the standard configuration settings. SingleBand.startup() is really a script and can
have components added and deleted.

In [ ]:
# configure a single band
single_band = SingleBand(S=S, cfg=cfg, band=band, auto_startup=True, verbose=verbose)
single_band.check_lock()

## Overbiases the detectors, an example of GroupedBiases
The GroupedBiases is a class that does the *same* operations to every `band` (bias line).

In [ ]:
# configure a collection of bands as a single bias group.
grouped_biases = GroupedBiases(S=S, cfg=cfg, bias_group=bias_group, verbose=verbose)
grouped_biases.overbias_tes(sleep_time=120, tes_bias=1)

## The Optimization - AutoTune
AutoTune is a class that operates on an abstract number of `bands`. This class acquires time streams,
then makes a map of white noise levels with respect to band and up conversion attenuation settings.
An acceptance function is uses to determine if subsequent tuning is needed, tuning is repeated for 5 loops
or until every band exits the tuning successfully.

To operate on a single `band`, have the iterable `bias_group` contain only a single element,
for example, `bias_group=[4]`.

The idea of tuning with multiple biases at a time is to minimize the number of time streams needed to
fully tune and array.

In [ ]:
# acquire time stream data
auto_tune = AutoTune(S=S, cfg=cfg, nperseg=nperseg, bias_group=bias_group, verbose=verbose)
auto_tune.tune_selector_up_atten(uc_attens_centers_per_band=None, loop_count_max=5,
                                 stream_time=stream_time, do_plots=False, fmin=5, fmax=50)
# print the plotting directory
print(f"plotting directory is:\n{S.plot_dir}")